In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
train_csv = pd.read_csv('C:/Users/admin/Downloads/emergency_data/train.csv')
train_csv.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [9]:
train_path = 'C:/Users/admin/Downloads/emergency_data/'
train_x, val_x, train_y, val_y = train_test_split(train_csv['image_names'], train_csv['emergency_or_not'], test_size = 0.1, random_state = 2, stratify=train_csv['emergency_or_not'])
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((1481,), (1481,)), ((165,), (165,)))

In [4]:
train_y.values

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [5]:
import os
import shutil

In [23]:
train_or_val = 'val'
if train_or_val == 'train':
    for file,val in zip(train_x.values, train_y.values):
        if val == 0:
            shutil.copyfile(train_path+'images/'+file, train_path+'train/non_emergency/'+file)
        else:
            shutil.copyfile(train_path+'images/'+file, train_path+'train/emergency/'+file)
else:
    for file,val in zip(val_x.values, val_y.values):
        if val == 0:
            shutil.copyfile(train_path+'images/'+file, train_path+'val/non_emergency/'+file)
        else:
            shutil.copyfile(train_path+'images/'+file, train_path+'val/emergency/'+file)

In [27]:
test_csv = pd.read_csv('C:/Users/admin/Downloads/emergency_data/test_vc2kHdQ.csv')
test_csv.head()
for name in test_csv['image_names'].values:
    source, dest = train_path+'images/'+name, train_path+'test/'+name
#     print(source, "   ", dest)
    shutil.copyfile(source, dest)

In [29]:
from tensorflow.keras.applications.Xception import Xception
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

ModuleNotFoundError: No module named 'tensorflow.keras.applications.Xception'

In [40]:
pre_trained_model = tf.keras.applications.Xception(input_shape = (299,299,3),
    include_top=True,
    weights="imagenet"
)

# # pre_trained_model.load_weights(weight_file)
# for layer in pre_trained_model.layers:
#       layer.trainable = False
# last_layer = pre_trained_model.get_layer('mixed7')
# last_output = last_layer.output

In [44]:
for layer in pre_trained_model.layers:
      layer.trainable = False
last_layer = pre_trained_model.get_layer('add_20')
last_output = last_layer.output

In [43]:
pre_trained_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation='relu')(x)

x = layers.Dropout(0.2)(x)

x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

In [38]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
      rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/training-data/training/training/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=128,
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/kaggle/input/training-data/testing/testing/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')